In [1]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12574489412141566286]

In [21]:
import platform
platform.platform()

'Windows-10-10.0.18362-SP0'

In [26]:
import os

pcm_path = ".//ETRI123//9_voice_download_4_Korean_English_by_Korean_part2"
wav_path = ".//ETRI//waves"

if not os.path.exists(wav_path):
    os.mkdir(wav_path)

print(sorted(os.listdir(pcm_path))[:5])
print(len(sorted(os.listdir(pcm_path)))) # # of speakers

['GSF03031LJH0', 'GSF03732JJH0', 'GSF04122LHK0', 'GSF04524CJI0', 'GSF04623LWJ0']
50


In [27]:
a = 0
for (root, dirs, files) in os.walk(pcm_path):
    a += len([filename for filename in files if filename.endswith(".pcm")])

In [28]:
# # of utterances
a

5000

In [29]:
import wave

for i, folder_path in enumerate(sorted(os.listdir(pcm_path))): # sorted!!!
    file_list = os.listdir(os.path.join(pcm_path, folder_path)) # label's path
    file_list_pcm = [files for files in file_list if files.endswith(".pcm")] # *.pcm files

    for j, file_path in enumerate(sorted(file_list_pcm)): # sorted!!!
        with open(os.path.join(pcm_path, folder_path, file_path), "rb") as pcm_file:
            pcm_data = pcm_file.read()
        # save as {label}_{features}.wav format
        with wave.open(os.path.join(wav_path, "{}_{}.wav".format(i, j)), 'wb') as wav_file:
            wav_file.setparams((1, 2, 16000, 0, 'NONE', 'NONE')) # 16kHz, mono channel
            wav_file.writeframes(pcm_data)

# print samples
print(os.listdir(wav_path)[:5])
print(len(os.listdir(wav_path)))

['0_0.wav', '0_1.wav', '0_10.wav', '0_11.wav', '0_12.wav']
5000


In [30]:
import numpy as np

num_labels = 50
num_features = 100
data_length = 3 ** 9

train_rate = 0.8
num_train_data = int(num_features * train_rate) # 80

In [37]:
# save features as *.npz format

from scipy.io import wavfile

# save dev files
for i in range(num_labels):
    
    data = np.zeros((num_train_data, data_length))
    
    for j in range(num_train_data):
        file_name = "{}_{}.wav".format(i, j)
        _, samples = wavfile.read(os.path.join(wav_path, file_name))
        
        start = int(len(samples) - data_length) // 2
        end = start + data_length
        
        data[j] = samples[start:end]
    
    np.savez_compressed(os.path.join("./ETRI/dev", "id{}".format(i)), data)

# save test files    
for i in range(num_labels):
    
    data = np.zeros((num_features - num_train_data, data_length))
    
    for j in range(num_features - num_train_data):
        file_name = "{}_{}.wav".format(i, j)
        _, samples = wavfile.read(os.path.join(wav_path, file_name))
        
        start = int(len(samples) - data_length) // 2
        end = start + data_length
        
        data[j] = samples[start:end]
    
    np.savez_compressed(os.path.join("./ETRI/test", "id{}".format(i)), data)

In [38]:
data.shape

(20, 19683)